In [1]:
import sys
import os
import warnings

# Ignore warnings
warnings.filterwarnings("ignore")

In [2]:
!{sys.executable} -m pip install tensorflow

In [3]:
import pandas as pd
import numpy as np

from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [6]:
# Load 22 and 23 data
df_filled_22 = pd.read_pickle(r'D:\capstone_F24\data\Cleaned_data\df_filled_22.pkl')
df_filled_23 = pd.read_pickle(r'D:\capstone_F24\data\Cleaned_data\df_filled_23.pkl')

In [8]:
df_filled_23

timestamp,2023-02-01 00:00:00,2023-02-01 01:00:00,2023-02-01 02:00:00,2023-02-01 03:00:00,2023-02-01 04:00:00,2023-02-01 05:00:00,2023-02-01 06:00:00,2023-02-01 07:00:00,2023-02-01 08:00:00,2023-02-01 09:00:00,...,2023-12-31 14:00:00,2023-12-31 15:00:00,2023-12-31 16:00:00,2023-12-31 17:00:00,2023-12-31 18:00:00,2023-12-31 19:00:00,2023-12-31 20:00:00,2023-12-31 21:00:00,2023-12-31 22:00:00,2023-12-31 23:00:00
station_complex_id,,,,,,,,,,,,,,,,,,,,,
1,27.0,16.0,15.0,16.0,47.0,227.0,654.0,1528.0,2208.0,1020.0,...,383.0,334.0,334.0,357.0,406.0,307.0,335.0,319.0,234.0,63.0
10,167.0,46.0,26.0,10.0,8.0,25.0,84.0,222.0,490.0,399.0,...,1.0,4.0,3.0,1.0,4.0,2.0,5.0,3.0,5.0,2.0
100,13.0,2.0,1.0,2.0,6.0,15.0,75.0,141.0,281.0,161.0,...,81.0,66.0,60.0,50.0,71.0,74.0,101.0,84.0,54.0,26.0
101,59.0,30.0,13.0,6.0,18.0,61.0,230.0,635.0,1223.0,765.0,...,353.0,364.0,376.0,364.0,290.0,318.0,302.0,270.0,217.0,107.0
118,94.0,25.0,15.0,3.0,4.0,9.0,49.0,95.0,188.0,114.0,...,212.0,231.0,249.0,236.0,292.0,286.0,309.0,320.0,326.0,148.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,15.0,3.0,6.0,13.0,39.0,87.0,233.0,566.0,768.0,433.0,...,193.0,197.0,147.0,143.0,182.0,191.0,164.0,141.0,114.0,55.0
96,13.0,6.0,5.0,8.0,24.0,72.0,193.0,421.0,621.0,364.0,...,175.0,157.0,121.0,136.0,153.0,214.0,159.0,139.0,105.0,51.0
97,31.0,14.0,19.0,22.0,37.0,118.0,290.0,871.0,1367.0,837.0,...,337.0,303.0,282.0,301.0,289.0,341.0,265.0,338.0,207.0,122.0


In [7]:
df_filled_22

timestamp,2022-02-01 00:00:00,2022-02-01 01:00:00,2022-02-01 02:00:00,2022-02-01 03:00:00,2022-02-01 04:00:00,2022-02-01 05:00:00,2022-02-01 06:00:00,2022-02-01 07:00:00,2022-02-01 08:00:00,2022-02-01 09:00:00,...,2022-12-31 14:00:00,2022-12-31 15:00:00,2022-12-31 16:00:00,2022-12-31 17:00:00,2022-12-31 18:00:00,2022-12-31 19:00:00,2022-12-31 20:00:00,2022-12-31 21:00:00,2022-12-31 22:00:00,2022-12-31 23:00:00
station_complex_id,,,,,,,,,,,,,,,,,,,,,
1,17.0,5.0,6.0,12.0,44.0,143.0,405.0,735.0,950.0,512.0,...,426.0,402.0,304.0,338.0,322.0,350.0,309.0,293.0,168.0,75.0
10,100.0,24.0,7.0,8.0,8.0,14.0,43.0,98.0,175.0,182.0,...,3.0,4.0,4.0,4.0,3.0,4.0,5.0,8.0,5.0,2.0
100,10.0,3.0,0.0,3.0,5.0,17.0,46.0,68.0,106.0,66.0,...,68.0,69.0,69.0,55.0,58.0,61.0,67.0,52.0,56.0,25.0
101,26.0,10.0,9.0,3.0,20.0,49.0,124.0,245.0,394.0,247.0,...,286.0,316.0,332.0,269.0,239.0,246.0,239.0,209.0,179.0,104.0
118,21.0,11.0,9.0,3.0,1.0,9.0,14.0,61.0,75.0,54.0,...,249.0,279.0,245.0,245.0,287.0,273.0,278.0,281.0,245.0,115.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,6.0,2.0,5.0,6.0,24.0,85.0,135.0,296.0,298.0,196.0,...,227.0,147.0,160.0,143.0,172.0,183.0,170.0,151.0,116.0,62.0
96,11.0,4.0,4.0,7.0,15.0,58.0,113.0,214.0,244.0,169.0,...,194.0,153.0,112.0,128.0,137.0,175.0,181.0,153.0,128.0,61.0
97,13.0,12.0,8.0,10.0,39.0,91.0,229.0,357.0,472.0,317.0,...,276.0,247.0,231.0,198.0,261.0,234.0,314.0,243.0,180.0,83.0


In [11]:
# Define a function to calculate symmetric mean absolute percentage error
def symmetric_mape(y_true, y_pred):
    return np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred))) * 100

In [12]:
# Parameters
batch_size = 10  # Set the number of stations to process per batch
predictions_file = 'df_23_pred_LSTM.pkl'
metrics_file = 'df_timeseries_metrics_LSTM.pkl'

In [13]:
# Initialize or load DataFrames for predictions and metrics
if os.path.exists(predictions_file):
    df_23_pred = pd.read_pickle(predictions_file)
else:
    df_23_pred = pd.DataFrame(columns=['station_complex_id', 'predicted_ridership_2023'])

if os.path.exists(metrics_file):
    df_timeseries_metrics = pd.read_pickle(metrics_file)
else:
    df_timeseries_metrics = pd.DataFrame(columns=['station_complex_id', 'rmse', 'mae', 'smape'])

In [14]:
# Total number of stations for tracking progress
total_stations = len(df_filled_22.index)
start_idx = len(df_23_pred)  # Continue from where the last batch stopped

In [15]:
# Function to prepare data for LSTM (create sequences)
def create_sequences(data, sequence_length=24):
    sequences = []
    labels = []
    for i in range(len(data) - sequence_length):
        sequences.append(data[i:i+sequence_length])
        labels.append(data[i+sequence_length])
    return np.array(sequences), np.array(labels)


In [18]:
# Process each batch of stations
for idx in range(start_idx, total_stations, batch_size):
    batch_end = min(idx + batch_size, total_stations)
    print(f"Processing stations {idx + 1} to {batch_end} ({(batch_end / total_stations) * 100:.2f}% complete)")
    
    # Temporary lists to store batch results before concatenation
    batch_pred = []
    batch_metrics = []
    
    # Process each station in the batch
    for station_id in df_filled_22.index[idx:batch_end]:
        # Extract the time series for this station from 2022 and 2023 data
        train_data = df_filled_22.loc[station_id].transpose().reset_index()
        train_data.columns = ['transit_timestamp', 'ridership']
        test_data = df_filled_23.loc[station_id].transpose().reset_index()
        test_data.columns = ['transit_timestamp', 'ridership']
        
        # Convert 'timestamp' to datetime and set it as the index
        train_data['transit_timestamp'] = pd.to_datetime(train_data['transit_timestamp'])
        test_data['transit_timestamp'] = pd.to_datetime(test_data['transit_timestamp'])
        
        # Directly use the raw ridership data without log transformation
        ridership_train = train_data['ridership'].values
        ridership_test = test_data['ridership'].values
        
        # Normalize the data using MinMaxScaler (scale between 0 and 1)
        scaler = MinMaxScaler(feature_range=(0, 1))
        ridership_train_scaled = scaler.fit_transform(ridership_train.reshape(-1, 1))
        ridership_test_scaled = scaler.transform(ridership_test.reshape(-1, 1))

        # Prepare data for LSTM (sequence length = 24 hours for hourly prediction)
        X_train, y_train = create_sequences(ridership_train_scaled, sequence_length=24)
        X_test, y_test = create_sequences(ridership_test_scaled, sequence_length=24)
        
        # Reshape data for LSTM input (samples, time steps, features)
        X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
        X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
        
        # Define the LSTM model
        model = Sequential()
        model.add(LSTM(units=50, return_sequences=False, input_shape=(X_train.shape[1], 1)))
        model.add(Dense(1))  # Output layer to predict next value
        
        # Compile the model
        model.compile(optimizer='adam', loss='mean_squared_error')
        
        # Train the LSTM model
        model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)
        
        # Forecast for the length of the test data
        forecast_values_scaled = model.predict(X_test)
        
        # Inverse transform the forecasted values to return them to original scale
        forecast_values = scaler.inverse_transform(forecast_values_scaled)
        
        # Add predictions to batch results
        batch_pred.append({
            'station_complex_id': station_id,
            'predicted_ridership_2023': forecast_values.tolist(),
            'predicted_timestamps': test_data['transit_timestamp'].iloc[24:].tolist()  # Adjust for sequence length
        })
        
        # Compute metrics
        rmse = np.sqrt(mean_absolute_error(test_data['ridership'].iloc[24:], forecast_values)**2)  # RMSE
        mae = mean_absolute_error(test_data['ridership'].iloc[24:], forecast_values)
        
        # Add metrics to batch results
        batch_metrics.append({
            'station_complex_id': station_id,
            'rmse': rmse,
            'mae': mae
        })
    
    # Concatenate batch results with main DataFrames
    df_23_pred = pd.concat([df_23_pred, pd.DataFrame(batch_pred)], ignore_index=True)
    df_timeseries_metrics = pd.concat([df_timeseries_metrics, pd.DataFrame(batch_metrics)], ignore_index=True)
    
    # Save the current state of predictions and metrics DataFrames
    df_23_pred.to_pickle(predictions_file)
    df_timeseries_metrics.to_pickle(metrics_file)
    print(f"Batch {idx + 1} to {batch_end} saved to pickle.")


Processing stations 1 to 10 (2.77% complete)
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step
Batch 1 to 10 saved to pickle.
Processing stations 11 to 20 (5.54% complete)
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step
250/250 ━━━━━━━━━

In [19]:
# Save metrics and predictions as desired
df_23_pred.to_pickle('df_23_pred_LSTM.pkl')
df_timeseries_metrics.to_pickle('df_timeseries_metrics_LSTM.pkl')


In [20]:
df_23_pred

,station_complex_id,predicted_ridership_2023,predicted_timestamps
0,1,"[[51.460418701171875], [37.575199127197266], [...","[2023-02-02 00:00:00, 2023-02-02 01:00:00, 202..."
1,10,"[[85.66735076904297], [35.20859146118164], [-9...","[2023-02-02 00:00:00, 2023-02-02 01:00:00, 202..."
2,100,"[[7.264353275299072], [8.049354553222656], [-0...","[2023-02-02 00:00:00, 2023-02-02 01:00:00, 202..."
3,101,"[[41.19313430786133], [30.327926635742188], [1...","[2023-02-02 00:00:00, 2023-02-02 01:00:00, 202..."
4,118,"[[102.69548797607422], [29.632213592529297], [...","[2023-02-02 00:00:00, 2023-02-02 01:00:00, 202..."
...,...,...,...
356,95,"[[-12.225086212158203], [-2.7507498264312744],...","[2023-02-02 00:00:00, 2023-02-02 01:00:00, 202..."
357,96,"[[2.718946695327759], [9.364445686340332], [7....","[2023-02-02 00:00:00, 2023-02-02 01:00:00, 202..."
358,97,"[[0.9972857236862183], [-4.06417179107666], [-...","[2023-02-02 00:00:00, 2023-02-02 01:00:00, 202..."
359,98,"[[23.597551345825195], [24.310914993286133], [...","[2023-02-02 00:00:00, 2023-02-02 01:00:00, 202..."


In [21]:
df_timeseries_metrics

,station_complex_id,rmse,mae,smape
0,1,54.684853,54.684853,NaN
1,10,118.049347,118.049347,NaN
2,100,14.551531,14.551531,NaN
3,101,54.605437,54.605437,NaN
4,118,31.978564,31.978564,NaN
...,...,...,...,...
356,95,25.658832,25.658832,NaN
357,96,22.248911,22.248911,NaN
358,97,62.191417,62.191417,NaN
359,98,31.496438,31.496438,NaN
